In [1]:
import numpy as np
import random
%run mnist_loader.ipynb

In [2]:
class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""

        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {} : {} / {}".format(j,self.evaluate(test_data),n_test));
            else:
                print("Epoch {} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

In [3]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

# Training set

The training data set is a list of tuples, each tuple has two ndarrays, the first one contains the pixel values of the image, the second one indicates the digit [0,1,2,3,4,5,6,7,8 or 9].

Here are the first 6 digits from the MNIST training data set:

![title](img/mnist.png)

In [4]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)

In [5]:
#Training data is a list of ndarrays,
type(training_data)

list

In [6]:
type(training_data[0])

tuple

In [7]:
training_data[0]

(array([[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0

In [8]:
#Pixel values of the first image
print(training_data[0][0])

[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.  

In [9]:
#Digit of the first image
print(training_data[0][1])

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [10]:
#Pixel values of the second image
print(training_data[1][0])

[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.  

In [11]:
#Digit of the second image
print(training_data[1][1])

[[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


# Training

In [12]:
net = Network([784, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0 : 9044 / 10000
Epoch 1 : 9190 / 10000
Epoch 2 : 9309 / 10000
Epoch 3 : 9363 / 10000
Epoch 4 : 9392 / 10000
Epoch 5 : 9401 / 10000
Epoch 6 : 9403 / 10000
Epoch 7 : 9446 / 10000
Epoch 8 : 9456 / 10000
Epoch 9 : 9454 / 10000
Epoch 10 : 9468 / 10000
Epoch 11 : 9480 / 10000
Epoch 12 : 9460 / 10000
Epoch 13 : 9459 / 10000
Epoch 14 : 9468 / 10000
Epoch 15 : 9462 / 10000
Epoch 16 : 9479 / 10000
Epoch 17 : 9477 / 10000
Epoch 18 : 9510 / 10000
Epoch 19 : 9508 / 10000
Epoch 20 : 9506 / 10000
Epoch 21 : 9489 / 10000
Epoch 22 : 9525 / 10000
Epoch 23 : 9504 / 10000
Epoch 24 : 9488 / 10000
Epoch 25 : 9528 / 10000
Epoch 26 : 9513 / 10000
Epoch 27 : 9526 / 10000
Epoch 28 : 9529 / 10000
Epoch 29 : 9523 / 10000


# The following code is not meant to run as a stand a lone version,
# rather it is used to illustrate each method in the NN class

# \__init__(self, sizes)
Objective: Initialize number of layers, biases and weights

Example, initialize the following network.
![title](img/network.png)

A total of 3 layers are required, hence the len of the size array should be 3. There should be 3, 4 and 2 nodes in each layer respectively, hence the "sizes" list should be [3,4,2]

In [4]:
nn = Network([3,4,2])
print("Number of layers = " + str(nn.num_layers))

Number of layers = 3


All nodes, excluding the ones from the input layer, should have biases assign to them.

self.biases = [np.random.randn(y,1) for y in sizes[1:]] 

Creates a list of ndarray that contains the biases for each neuron in each layer. 


![title](img/biases.png)

In [5]:
layers = [3,4,2]
biases = [np.random.randn(1,y) for y in layers[1:]]
biases

[array([[ 0.31779313,  0.0551198 ,  1.02832841, -0.87395342]]),
 array([[ 0.9157837, -0.3465925]])]

Create the connections between layers

![title](img/weights.png)

$w^{l}_{jk}$ is the weight from the k neuron in layer (l-1) to the jth neuron in layer l

self.weights = [np.random.randn(y,x) for x,y in zip(layers[:-1],layers[1:])]

In [6]:
#(nodes_layer_l,nodes_layer_l+1)
list(zip(layers[:-1],layers[1:]))

[(3, 4), (4, 2)]

In [7]:
#Generate random values for the weights.
#Connect the 2 nodes from layer 3 to the 4 nodes of layer 2
np.random.randn(2,4)

array([[ 0.59060842, -0.28383816, -1.38692659,  1.41066409],
       [-0.50376328, -2.08294898,  1.5993013 ,  0.50585564]])

In [8]:
weights = [np.random.randn(y,x) for x,y in zip(layers[:-1],layers[1:])]
weights

[array([[ 0.03749198,  0.07428665,  0.85117139],
        [-1.13256801, -0.48958943,  2.17431206],
        [ 0.08705631,  1.17192449, -1.75764294],
        [ 0.23686487, -0.93539366, -0.03047311]]),
 array([[ 1.69973585,  0.11518644,  1.82775156,  0.62015731],
        [-0.14629818,  1.00296349, -1.63425238,  0.5107543 ]])]

# SGD()

Generate mini batches

![title](img/mini_batch.png)

In [9]:
batch_size_example = 4
sequence = list(range(10))
n_example = len(sequence)
random.shuffle(sequence)
mini_batches_example = [sequence[k:k+batch_size_example] for k in range(0,n_example,batch_size_example)]
mini_batches_example

[[7, 5, 3, 9], [8, 6, 0, 4], [1, 2]]

In [10]:
batch_size = 4
features = [(x,x,x) for x in range(10)]
labels = [(3*x)%2 for x in range(10)]
dataset = [z for z in zip(features,labels)]
n = len(features)
random.shuffle(dataset)

In [11]:
dataset

[((0, 0, 0), 0),
 ((1, 1, 1), 1),
 ((9, 9, 9), 1),
 ((5, 5, 5), 1),
 ((4, 4, 4), 0),
 ((7, 7, 7), 1),
 ((2, 2, 2), 0),
 ((6, 6, 6), 0),
 ((8, 8, 8), 0),
 ((3, 3, 3), 1)]

In [12]:
mini_batches = [dataset[k:k+batch_size] for k in range(0,n,batch_size)]

In [13]:
mini_batch = mini_batches[0]
mini_batches

[[((0, 0, 0), 0), ((1, 1, 1), 1), ((9, 9, 9), 1), ((5, 5, 5), 1)],
 [((4, 4, 4), 0), ((7, 7, 7), 1), ((2, 2, 2), 0), ((6, 6, 6), 0)],
 [((8, 8, 8), 0), ((3, 3, 3), 1)]]

In [14]:
mini_batch

[((0, 0, 0), 0), ((1, 1, 1), 1), ((9, 9, 9), 1), ((5, 5, 5), 1)]

# Backpropagation

Consist of two phases:

Forward pass:
Inputs are passed through the network and output predictions are obtained


Backward pass:
Compute the gradient of the loss function at the final layer and update the weights backwards

# feedforward(self,a)

$$z = w⋅x+b$$

![title](img/sigmoid_neurons.png)


$$σ(z)=\frac{1}{1-e^{-z}}$$


![title](img/sigmoid.png)

$$a = sigmoid(np.dot(w,a)+b)$$

![title](img/dataflow.png)

![title](img/backprop.jpg)

In [25]:
def backprop(x,y):

    nabla_b = [np.zeros(b.shape) for b in biases]
    nabla_w = [np.zeros(w.shape) for w in weights]
    
    #Feedforward
    activation = x
    activations = [x]
    zs = []
    for b, w in zip(biases,weights):
        z = np.dot(w,activation)
        zs.append(z)
        activation = sigmoid(z)
        activations.append(activation)
    delta = cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
    nabla_b[-1] = delta
    nabla_w[-1] = np.dot(delta, activations[-2].transpose())
    for l in range(2, self.num_layers):
        z = zs[-l]
        sp = sigmoid_prime(z)
        delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
        nabla_b[-l] = delta
        nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
    return (nabla_b, nabla_w)

In [23]:
def cost_derivative(output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

# update_mini_batch(mini_batch,eta)

In [17]:
biases[0].shape

(1, 4)

In [18]:
biases[1].shape

(1, 2)

In [19]:
[np.zeros(b.shape) for b in biases]

[array([[0., 0., 0., 0.]]), array([[0., 0.]])]

In [20]:
weights[0].shape

(4, 3)

In [21]:
[np.zeros(w.shape) for w in weights]

[array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]), array([[0., 0., 0., 0.],
        [0., 0., 0., 0.]])]

![title](img/update_parameters.png)

In [ ]:
for x,y in mini_batch:
    delta_nabla_b, delta_nabla_w = backprop(x,y)
    nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
    nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
    self.weights = [w-(eta/len(mini_batch))*nw
                   for w, nw in zip(self.weights, nabla_w)]
    self.biases = [b-(eta/len(mini_batch))*nb
                   for b, nb in zip(self.biases, nabla_b)]

# Sigmoid

In [29]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

$$σ(z)=\frac{1}{1-e^{-z}}$$

$$\frac{d}{dx}σ(z)=\frac{1}{1-e^{-z}}*(1-\frac{1}{1-e^{-z}})$$

![title](img/sigmoid_dsigmoid.png)